## Data Scrape

**Chad P. Tsikalas**

**Email: [cpt268@stern.nyu.edu](cpt268@stern.nyu.edu)**

The following code was used to scrape Spotrac to acquire the salaries of all NFL players in the 2017 season. This scrape provided the raw data that was used to complete my project.

Because of an unfamiliarity with scraping and BeautifulSoup, a Google search led me to a similar scrape of Spotrac from which this code was closely derived. For transparency, that guide can be found [here](https://wcontractor.github.io/nfl-salary-part2.html).

In [2]:
import pandas as pd
import bs4
import urllib
import numpy as np
from bs4 import BeautifulSoup

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
base_url = "http://www.spotrac.com/nfl/cap/2017/"

def get_page(url):
    page = urlopen(base_url)
    soup = BeautifulSoup(page, 'lxml')
    file = open("spotrac_links.txt", 'w')
    file.write(str(soup))
    file.close()

def get_team_table(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, 'lxml')

In [17]:
get_page(base_url)
with open("spotrac_links.txt", 'r') as file:
    for line in file:
        line = line.strip()
from bs4 import BeautifulSoup
page = open("spotrac_links.txt", 'r')
soup = BeautifulSoup(page, "lxml")
div = soup.find("div","teams")
import re
links = div.find_all('a')

In [5]:
from urllib.request import urlopen
def get_team_table(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, 'lxml')
    data_rows = [row for row in soup.find("table", "datatable").find_all("tr")]
    return data_rows

In [6]:
team_data = []
for link in links:
    team_data.append(get_team_table(link.get('href')))

In [7]:
table_data = []
for row in team_data:
    for element in row:
        if soup.find_all("td", attrs={"class":" right xs-hide "}) is not None:
            table_data.append(element.get_text())

player_data = []
for row in table_data:
    player_data.append(row.split("\n"))
    print(player_data)

len(player_data)    

[['', 'Active Players  (53)', 'Pos.', 'Base Salary', 'Signing Bonus', 'Roster Bonus', 'Option Bonus', 'Workout Bonus', 'Restruc. Bonus', 'Misc.', 'Dead Cap', 'Cap Hit', 'Cap %', '']]
[['', 'Active Players  (53)', 'Pos.', 'Base Salary', 'Signing Bonus', 'Roster Bonus', 'Option Bonus', 'Workout Bonus', 'Restruc. Bonus', 'Misc.', 'Dead Cap', 'Cap Hit', 'Cap %', ''], ['', 'Bitonio', 'Joel Bitonio ', 'G', '$3,164,777', '$573,036', '$8,500,000', '-', '-', '-', '-', '($17,737,813)', '$12,237,813 ', '7.81', '']]
[['', 'Active Players  (53)', 'Pos.', 'Base Salary', 'Signing Bonus', 'Roster Bonus', 'Option Bonus', 'Workout Bonus', 'Restruc. Bonus', 'Misc.', 'Dead Cap', 'Cap Hit', 'Cap %', ''], ['', 'Bitonio', 'Joel Bitonio ', 'G', '$3,164,777', '$573,036', '$8,500,000', '-', '-', '-', '-', '($17,737,813)', '$12,237,813 ', '7.81', ''], ['', 'Zeitler', 'Kevin Zeitler ', 'G', '$6,000,000', '$2,400,000', '-', '-', '-', '-', '-', '($23,000,000)', '$8,400,000 ', '5.36', '']]
[['', 'Active Players  (53

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


1726

In [8]:
df = pd.DataFrame(player_data)
df = df.drop(14, 1)
df = df.drop(0, 1)
df = df.drop(1, 1)

df = df.drop(df.index[[0]])
print(df.shape)
df.head()

(1725, 12)


,2,3,4,5,6,7,8,9,10,11,12,13
1,Joel Bitonio,G,"$3,164,777","$573,036","$8,500,000",-,-,-,-,"($17,737,813)","$12,237,813",7.81
2,Kevin Zeitler,G,"$6,000,000","$2,400,000",-,-,-,-,-,"($23,000,000)","$8,400,000",5.36
3,Myles Garrett,DE,"$465,000","$5,064,501",-,-,-,-,-,"($30,412,255)","$5,529,501",3.53
4,Christian Kirksey,ILB,"$3,797,000","$1,365,625",-,-,-,-,-,"($14,962,625)","$5,162,625",3.30
5,J.C. Tretter,C,"$2,000,000","$1,500,000","$109,375",-,-,-,-,"($6,500,000)","$3,609,375",2.30


In [9]:
players = []
for row in team_data[0]:
    if row.get_text("tr") is not None:
        players.append(row) 

column_headers = [col.get_text() for col in players[0].find_all("th") if col.get_text()]
len(column_headers)

12

In [10]:
df.columns = column_headers
df.head()

,Active Players (53),Pos.,Base Salary,Signing Bonus,Roster Bonus,Option Bonus,Workout Bonus,Restruc. Bonus,Misc.,Dead Cap,Cap Hit,Cap %
1,Joel Bitonio,G,"$3,164,777","$573,036","$8,500,000",-,-,-,-,"($17,737,813)","$12,237,813",7.81
2,Kevin Zeitler,G,"$6,000,000","$2,400,000",-,-,-,-,-,"($23,000,000)","$8,400,000",5.36
3,Myles Garrett,DE,"$465,000","$5,064,501",-,-,-,-,-,"($30,412,255)","$5,529,501",3.53
4,Christian Kirksey,ILB,"$3,797,000","$1,365,625",-,-,-,-,-,"($14,962,625)","$5,162,625",3.30
5,J.C. Tretter,C,"$2,000,000","$1,500,000","$109,375",-,-,-,-,"($6,500,000)","$3,609,375",2.30


In [11]:
df

,Active Players (53),Pos.,Base Salary,Signing Bonus,Roster Bonus,Option Bonus,Workout Bonus,Restruc. Bonus,Misc.,Dead Cap,Cap Hit,Cap %
1,Joel Bitonio,G,"$3,164,777","$573,036","$8,500,000",-,-,-,-,"($17,737,813)","$12,237,813",7.81
2,Kevin Zeitler,G,"$6,000,000","$2,400,000",-,-,-,-,-,"($23,000,000)","$8,400,000",5.36
3,Myles Garrett,DE,"$465,000","$5,064,501",-,-,-,-,-,"($30,412,255)","$5,529,501",3.53
4,Christian Kirksey,ILB,"$3,797,000","$1,365,625",-,-,-,-,-,"($14,962,625)","$5,162,625",3.30
5,J.C. Tretter,C,"$2,000,000","$1,500,000","$109,375",-,-,-,-,"($6,500,000)","$3,609,375",2.30
6,Jamar Taylor,CB,"$2,500,000","$750,000",-,-,"$25,000",-,-,"($4,775,000)","$3,275,000",2.09
7,Danny Shelton,DT,"$1,498,970","$1,692,939",-,-,-,-,-,"($6,915,802)","$3,191,909",2.04
8,Britton Colquitt,P,"$1,650,000",-,"$1,500,000",-,-,-,-,"($5,700,000)","$3,150,000",2.01
9,Isaiah Crowell,RB,"$2,746,000",-,-,-,-,-,-,-,"$2,746,000",1.75
10,Corey Coleman,WR,"$979,773","$1,669,090",-,-,-,-,-,"($9,535,907)","$2,648,863",1.69


In [12]:
rows_to_be_dropped = df.loc[df['Cap Hit'] == 'Cap %'].index
df = df.drop(rows_to_be_dropped)

In [13]:
df['Cap Hit'] = df['Cap Hit'].str.replace("$","")
df['Cap Hit'] = df['Cap Hit'].str.replace(",","")

In [18]:
df['Cap Hit'].replace(to_replace = ['- '], value = np.nan, inplace = True)
df['Cap Hit'] = df['Cap Hit'].astype("float")
df.head(25)

,Active Players (53),Pos.,Base Salary,Signing Bonus,Roster Bonus,Option Bonus,Workout Bonus,Restruc. Bonus,Misc.,Dead Cap,Cap Hit,Cap %
1,Joel Bitonio,G,"$3,164,777","$573,036","$8,500,000",-,-,-,-,"($17,737,813)",12237813.0,7.81
2,Kevin Zeitler,G,"$6,000,000","$2,400,000",-,-,-,-,-,"($23,000,000)",8400000.0,5.36
3,Myles Garrett,DE,"$465,000","$5,064,501",-,-,-,-,-,"($30,412,255)",5529501.0,3.53
4,Christian Kirksey,ILB,"$3,797,000","$1,365,625",-,-,-,-,-,"($14,962,625)",5162625.0,3.30
5,J.C. Tretter,C,"$2,000,000","$1,500,000","$109,375",-,-,-,-,"($6,500,000)",3609375.0,2.30
6,Jamar Taylor,CB,"$2,500,000","$750,000",-,-,"$25,000",-,-,"($4,775,000)",3275000.0,2.09
7,Danny Shelton,DT,"$1,498,970","$1,692,939",-,-,-,-,-,"($6,915,802)",3191909.0,2.04
8,Britton Colquitt,P,"$1,650,000",-,"$1,500,000",-,-,-,-,"($5,700,000)",3150000.0,2.01
9,Isaiah Crowell,RB,"$2,746,000",-,-,-,-,-,-,-,2746000.0,1.75
10,Corey Coleman,WR,"$979,773","$1,669,090",-,-,-,-,-,"($9,535,907)",2648863.0,1.69


In [16]:
from pandas import ExcelWriter
writer = ExcelWriter("NFLSalaryData2017.xlsx")
df.to_excel(writer)
writer.save()